In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms, models
from torchvision.models import resnet50

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter
!pip install kagglehub[pandas-datasets]
!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
# Load the dataset's plates.csv
file_path = "new plates/plates.csv"

df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "gpiosenka/us-license-plates-image-classification",
  file_path,
)

print("First 5 records:", df.head())

In [ ]:
%cd drive/MyDrive/CS467\ Final\ Project/new\ plates/
!ls

: 

In [ ]:
# image input size: 128 X 224 X 3
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 224
RGB_NUM = 3

class CustomDataset(Dataset):
    # 0001.jpg, 0
    # 0001.jpg, [1,0,0,0,0,0,0, ...]
    def __init__(self, image_paths, labels):
        self.image_paths = image_paths
        self.labels = labels

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]

        # 1 hot encoding setup
        label_idx = self.labels[idx]
        label = torch.zeros(56)
        label[label_idx] = 1

        # Load the image
        image = Image.open(image_path)


        # Convert the image to a tensor
        to_tensor = transforms.ToTensor()
        tensor_image = to_tensor(image)

        return tensor_image, label

df_train = df[df["data set"] == "train"][["filepaths", "class id"]]
train_data = CustomDataset(df_train["filepaths"], df_train["class id"])

# print a sample image
print(train_data[5000][1])
print(train_data.__getitem__(0))

dl = DataLoader(train_data, batch_size=32, shuffle=True)


In [ ]:
# initialize resnet50 model
model = resnet50()